# Import medcat and load model pack....

### cat =...

## import concept description files

In [3]:
import pandas as pd
from tqdm import tqdm

In [ ]:
opc4s_map = pd.read_csv('../gloabl_files/vocab_folders/opc4/CONCEPT.csv', sep='\t')

opc4s_map = opc4s_map[opc4s_map['concept_code'] != 'OMOP generated']

opc4s_map = opc4s_map[opc4s_map['domain_id']!='Metadata']

opc4s_map

#####  Call medCat get entities on concept names, sort by meta annotations and accuracy. Store all results. Store preffered result (sorted)

In [ ]:


# Assuming you have the 'res' variable with the provided structure

data = []

code_list_name = list(opc4s_map['concept_name'])
code_list_code = list(opc4s_map['concept_code'])

for i in tqdm(range(0, len(code_list_name))):
    res = pat2vec_obj.cat.get_entities(code_list_name[i])
    
    if 'entities' in res and len(res['entities']) > 0:
        # Sort the annotations based on priority
        sorted_annotations = sorted(res['entities'].values(), key=lambda x: (x['types'], x['meta_anns']['Subject/Experiencer']['value'] == 'Patient', -x['acc']), reverse=False)
        
        # Take the first annotation as per the priority
        selected_annotation = sorted_annotations[0]
        
        # Create a dictionary for the row
        row_data = {
            'Code': code_list_name[i],
            'Pretty_Name': selected_annotation['pretty_name'],
            'CUI': selected_annotation['cui'],
            'Type': selected_annotation['types'][0] if selected_annotation['types'] else None,
            'Source_Value': selected_annotation['source_value'],
            'Detected_Name': selected_annotation['detected_name'],
            'Context_Similarity': selected_annotation['context_similarity'],
            'Start': selected_annotation['start'],
            'End': selected_annotation['end'],
            'Time': selected_annotation['meta_anns']['Time']['value'],
            'Presence': selected_annotation['meta_anns']['Presence']['value'],
            'Subject_Experiencer': selected_annotation['meta_anns']['Subject/Experiencer']['value'],
            'Acc': selected_annotation['acc'],
            'Found_annotations' : sorted_annotations
        }
        
        data.append(row_data)

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data)

# Display the DataFrame
df



In [ ]:
df.to_csv('../gloabl_files/medcat_ontology_maps/medcat_opc4s_map.csv')